# Self-Learning Contact Strategy Simulator

We're simulating a real-world debt recovery scenario:

- Each client needs to be contacted.
- You can try SMS, Email, or a Manual Call.
- Not every method works for every client.
- We’ll simulate how successful each method is depending on the client’s profile.

Over time, the system learns which methods work best first using Reinforcement Learning ideas.






In real debt recovery, we don't want to call everyone manually — it's expensive. But we also don’t want to miss recovery from someone who won’t reply to SMS/email.

 - - This simulator helps decide:
- - 

- Who should get an SMS first?
- Who is worth a direct call?
- Who can be left to an email reminder?

It’s like trial-and-error simulation , but with logic behind it.

CD Project can use this to:

- Test new strategies
- Save contact cost
- Prioritize contact efforts by likelihood of success

## Taking this as an RL Problem



1. State = Client profile (Segment score, contact info, etc.)
2. Actions = [SMS, Email, Call]
3. Rewards = Simulated chance of success ( defining probabilities for now)



A basic policy will be:

- Try SMS first.
- If failed, fallback to Email.
- If failed, do a Manual Call.

Eventually a smart agent could learn to skip email if the client is high risk and go straight to call — but starting with a rule-based simulator

In [1]:
import pandas as pd
import numpy as np
import random

In [ ]:


# dataset
df = pd.read_csv("/Users/rg/ACADEMICS/Interview/Connected Data Comapany/MAY/Dataset/Client_Embeddings_NN/clients_with_Kmeans_cluster_strategies.csv")

# fixed success probabilities - cause anyway it wil learn later
success_rates = {
    'SMS': {
        'Tier 1 - Low Risk': 0.8,
        'Tier 2 - Medium Risk': 0.6,
        'Tier 3 - High Risk': 0.3
    },
    'Email': {
        'Tier 1 - Low Risk': 0.5,
        'Tier 2 - Medium Risk': 0.4,
        'Tier 3 - High Risk': 0.2
    },
    'Call': {
        'Tier 1 - Low Risk': 0.7,
        'Tier 2 - Medium Risk': 0.7,
        'Tier 3 - High Risk': 0.5
    }
}

# Simulator 
def simulate_contact(uid, risk_tier):
    path = []
    for method in ['SMS', 'Email', 'Call']:
        path.append(method)
        prob = success_rates[method][risk_tier]
        if random.random() < prob:
            return {
                'UID': uid,
                'Risk Tier': risk_tier,
                'Final Action': method,
                'Contact Path': " → ".join(path),
                'Success': 1,
                'Comment': f'Success via {method}'
            }
    return {
        'UID': uid,
        'Risk Tier': risk_tier,
        'Final Action': 'None',
        'Contact Path': " → ".join(path),
        'Success': 0,
        'Comment': 'No contact success'
    }

# simulation to all clients
results = []
for _, row in df.iterrows():
    result = simulate_contact(row['UID'], row['Action_Tier'])
    results.append(result)

# dataframe
sim_df = pd.DataFrame(results)

# results
sim_df.to_csv("/Users/rg/ACADEMICS/Interview/Connected Data Comapany/MAY/Dataset/reinforcement_learning/contact_strategy_simulation_results.csv", index=False)

# sample output
print("Simulation complete! Here are the first few rows:\n")
print(sim_df.head(10))


Simulation complete! Here are the first few rows:

         UID          Risk Tier Final Action        Contact Path  Success  \
0  194477727  Tier 1 - Low Risk          SMS                 SMS        1   
1  384979263  Tier 1 - Low Risk         Call  SMS → Email → Call        1   
2  278387760  Tier 1 - Low Risk          SMS                 SMS        1   
3  793208927  Tier 1 - Low Risk         Call  SMS → Email → Call        1   
4  626258645  Tier 1 - Low Risk         Call  SMS → Email → Call        1   
5  768599736  Tier 1 - Low Risk        Email         SMS → Email        1   
6  193700333  Tier 1 - Low Risk          SMS                 SMS        1   
7  444355405  Tier 1 - Low Risk          SMS                 SMS        1   
8  644114704  Tier 1 - Low Risk          SMS                 SMS        1   
9  895570481  Tier 1 - Low Risk          SMS                 SMS        1   

             Comment  
0    Success via SMS  
1   Success via Call  
2    Success via SMS  
3   Succe

In [6]:
import pandas as pd


sim_df = pd.read_csv("/Users/rg/ACADEMICS/Interview/Connected Data Comapany/MAY/Dataset/reinforcement_learning/contact_strategy_simulation_results.csv")

# count summary 
summary = pd.crosstab(sim_df['Risk Tier'], sim_df['Final Action'])

# success rate per Risk Tier
success_rate = sim_df.groupby('Risk Tier')['Success'].mean().round(2) * 100
summary['Success Rate (%)'] = success_rate


# total row
summary.loc['Total'] = summary.sum(numeric_only=True)
summary.loc['Total', 'Success Rate (%)'] = (sim_df['Success'].mean() * 100).round(2)

# output
summary.to_csv("contact_strategy_summary_table.csv")

#summary
print(" RL Contact Strategy Summary Table:\n")
print(summary)


 RL Contact Strategy Summary Table:

Final Action           Call  Email    SMS  Success Rate (%)
Risk Tier                                                  
Tier 1 - Low Risk      15.0   17.0  174.0             98.00
Tier 2 - Medium Risk   15.0   11.0   49.0             89.00
Tier 3 - High Risk     82.0   37.0   80.0             74.00
Total                 112.0   65.0  303.0             85.41


1. Tier 1 (Low Risk) Clients

- Most responded to SMS (174/210 = 83%)
- Very few required escalation

- - Only 4 clients could not be reached at all  Shows that SMS-first is highly efficient for this group — low-cost, high success.

2. Tier 2 (Medium Risk) Clients

- More varied results:
- Only 58% recovered via SMS/Email.
- Call needed for 18% of clients.

- - 9 clients could not be reached  
- - Mixed-contact strategies work well — but seeing signs of resistance.





3. Tier 3 (High Risk) Clients

- SMS success drops a lot — only 30% respond to SMS.
- Call is the top success method: 82 clients saved via call

- - Still, 69 clients could not be reached 
- - Proves that high-risk clients often require escalation (manual contact), and even then success is not guaranteed.